In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
#gdis data 
gdis = pd.read_csv('../../data/pend-gdis-1960-2018-disasterlocations.csv')
#get emdat dataset
emdat = pd.read_csv('../../data/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])
gdis.head(2)

/var/folders/gp/hc93gpc55qs79hz6r0bfm83m0000gn/T/ipykernel_76199/610470096.py:4: DtypeWarning: Columns (8,16,17,18,19,24,25,26,27,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  emdat = pd.read_csv('../../data/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])


,id,country,iso3,gwno,year,geo_id,geolocation,level,adm1,adm2,adm3,location,historical,hist_country,disastertype,disasterno,latitude,longitude
0,109,Albania,ALB,339.0,2009,346,Ana E Malit,3,Shkoder,Shkodres,Ana E Malit,Ana E Malit,0,NaN,flood,2009-0631,42.020948,19.418317
1,109,Albania,ALB,339.0,2009,351,Bushat,3,Shkoder,Shkodres,Bushat,Bushat,0,NaN,flood,2009-0631,41.959294,19.514309


In [6]:
#select certain columns from emdat and join with gdis 
# we grab the disaster number and convert it into string format and grab everything except for the last 4 ( so only the dates)
emdat['disasterno'] = emdat['Dis No'].str[:-4] #format disasterno to merge  

#These are the columns that we want to keep  , the features below are left out!
cols = ['disasterno', 'Year', 'Event Name', 
#         'Disaster Type', 'Disaster Subtype', 
#         'Region', 'Continent', #'Location',
        'Start Year', 'Start Month', 'Start Day', 
        'End Year', 'End Month','End Day',  
        "Total Damages, Adjusted ('000 US$)"] 

emdat = emdat[cols]

#join emdat and gdis into one dataframe
gdis = pd.merge(emdat, gdis, on = 'disasterno', how='right')

gdis = gdis.drop_duplicates(subset=['id'])
print('shape', gdis.shape)

# New Grid IDs
#new grid_id: round to integers 
gdis['lat_grid'] = gdis['latitude'].round().astype(int)
gdis['lon_grid'] = gdis['longitude'].round().astype(int)
gdis['grid_id'] = list(zip(gdis['lat_grid'],gdis['lon_grid'])) 
print('total number of grid pairs', len(gdis.grid_id.value_counts()))

# check if they lie in range
print('lon range', gdis['lon_grid'].min(), gdis['lon_grid'].max()) 
print('lat range', gdis['lat_grid'].min(), gdis['lat_grid'].max()) 


shape (9924, 27)
total number of grid pairs 2827
lon range -178 180
lat range -54 68


In [7]:
# Construct X dataframe

#pivot function: change rows of info into tables 
def pivot(df_in, id_col='disastertype', id_list=['Flood']):
    #Drop and reset the index column
    df = df_in.reset_index(drop = True)

    # one set of disaster
    for id in id_list:
        #initialize columns
        df[id+'_bin'] = 0
        df[id+'_amt'] = 0
        df[id+'_ct'] = 0
        
        
        # so if the flood happens and we find it, we create these columns and get the flood costs
        df.loc[(df[id_col]==id), id+'_bin'] = 1
        df.loc[(df[id_col]==id), id+'_amt'] = df["Total Damages, Adjusted ('000 US$)"].astype(float)
        df.loc[(df[id_col]==id), id+'_ct'] = 1

    return df

# id_list= df_sub['Disaster Type'].unique()
id_list= gdis['disastertype'].unique().tolist()
print(id_list)
df_pivot= pivot(gdis, id_col = 'disastertype', id_list = id_list)


#aggregate columns by year
# We need to include the longitude and lattitude here 
def aggregate_yrly(df):
    #aggregate count
    col_ct = [col for col in df.columns if '_ct' in col]
    df_ct = df.groupby(['grid_id','year'])[col_ct].agg('sum')
    
    #aggregate amount 
    col_amt = [col for col in df.columns if '_amt' in col]
    df_amt = df.groupby(['grid_id','year'])[col_amt].agg('sum')
    
    #aggregate binary
    col_bin = [col for col in df.columns if '_bin' in col]
    df_bin= df.groupby(['grid_id','year'])[col_bin].agg('max')

    #join
    df1= pd.concat([df_amt, df_ct], axis=1)
    df_out = pd.concat([df1, df_bin], axis=1)
    
    return df_out.reset_index()



df_yrly = aggregate_yrly(df_pivot)
# df_yrly.to_csv('../../data/df_yrly.csv')

['flood', 'storm', 'earthquake', 'extreme temperature ', 'landslide', 'volcanic activity', 'drought', 'mass movement (dry)']


In [8]:
df_yrly = df_yrly.loc[df_yrly['year']>1979].copy().reset_index(drop=True)
print(len(df_yrly))

7977


In [9]:
# Construct Y
bin_col =  [col for col in df_yrly.columns if '_bin' in col]
df_yrly_bin = df_yrly[['grid_id','year'] + bin_col]


# get a list of grid_ids 
grid_id = gdis['grid_id'].unique()
# get a list of year information 
print(gdis.Year.min(), gdis.Year.max())
year_id = np.arange(1979, 2019, 1) 
#create multi-index: each grid id, spanning over the years 
idd = pd.MultiIndex.from_product([grid_id, year_id],
                           names=['grid_id', 'year'])

#length should be |years| * |grid_ids| 
print(len(idd))
#get dataframe 
idd = idd.to_frame().reset_index(drop=True)

#master disaster targets for all years and all ids: 
#merge with df_yrly 
y_master = pd.merge(idd, df_yrly_bin, on=['grid_id','year'], how='left').fillna(0)

#keep just the binary
# y_master.sum(axis=0)

# Filter data to previously flooded region
#step 1: filter xy_df to those grid_ids with previous frequent flooding history 
agg = df_yrly.groupby('grid_id').agg({'flood_bin':'sum'})
grid_id_ls = agg.loc[agg['flood_bin']>=2].index.tolist()
print('no of grid_ids selected', len(grid_id_ls))


#step 2: interpolate years to record all years, fill with 0 without any flood using idd 
#create multi-index: each grid id, spanning over the years 
year_id = np.arange(1979, 2019, 1) 
idd = pd.MultiIndex.from_product([grid_id_ls, year_id],
                           names=['grid_id', 'year'])

#length should be |years| * |grid_ids| 
print('len of idd', len(idd))
#get dataframe 
idd = idd.to_frame().reset_index(drop=True)



1960.0 2018.0
113080
no of grid_ids selected 791
len of idd 31640


In [122]:
# pwd
import sys
# setting path
sys.path.append('../../src')

#import model training module 
import models as m 

#split training and testing 
from sklearn.model_selection import train_test_split
# import shap
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

import pickle 
print(pickle.format_version)

4.0


In [130]:
#nlp_names = ['nlp_cls_transfer.pkl','nlp_avg.pkl','nlp_cls_finetune.pkl','nlp_cls_transfer.pkl','nlp_cls.pkl']


#load processed nlp features: 
#note: you can load other nlp features too, they all start with 'nlp_*' 
file = open('../../data/nlp_cls_transfer.pkl', 'rb') 
# load file
df_nlp = pickle.load(file)
# close the file
file.close()
df_nlp.shape

df_nlp = string_to_tuple(df_nlp, 'grid_id')

df_nlp = df_nlp.drop(['location','txt','label','flood_ct_x'], axis=1)


In [121]:
# running the model 

#attach target for a particular disease for next n years, using y_master  
#next_n is how we choose the next n-periods for the prediction target
def attach_target(x_df, y_master, disaster, next_n): 
    y = y_master.copy()
    #shift years
    y['year'] = y['year'] - next_n
    #keep for particular disaster 
    y = y[['grid_id','year',disaster+'_bin']]
    # Rename into target
    y = y.rename(columns={disaster +'_bin': 'target_' + disaster + '_'+ str(next_n)})
    xy_df = pd.merge(x_df, y, on = ['grid_id','year'], how='inner')
    return xy_df

#construct x_df with various modalities: stat, nlp, era features 

#select to those id pass the filtering criteria 
x_df = df_yrly.loc[df_yrly['grid_id'].isin(grid_id_ls)]
#interpolate missing years to have no flood 
x_df = pd.merge(idd, x_df, on=['grid_id','year'], how='left').fillna(0)
#add back lat and long info 
#x_df[['lat', 'lon']] = pd.DataFrame(x_df['grid_id'].tolist(), index=x_df.index) Is this even necessary?

print('length of x_df', len(x_df))


#construct xy_df, depending on the n_pred target period 
n_pred = 2

# x_df = x_df.loc[x_df['year']>=1979] #crop to after 1979 
xy_df = attach_target(x_df, y_master, 'flood', n_pred)
print('length of xy_df', len(xy_df))
print('imbalance', xy_df.filter(regex='target').sum()/len(xy_df))

#construct x,y train and test set 
x = xy_df.drop(xy_df.filter(regex='target').columns, axis=1)#drop target col 
x = x.select_dtypes(['number'])#drop index col
y = xy_df.filter(regex='target') #filter to cols containing target 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print(x_train.shape) #x_train contains stat features only 


y_pred, y_pred_prob = m.run_xgb(x_train, y_train, x_test)
sc_xgb = m.get_scores_clf(y_test, y_pred_prob)


length of x_df 31640
length of xy_df 30058
imbalance target_flood_2    0.091956
dtype: float64
(21040, 25)
running xgb...
Train AUC:  0.6886016465616585
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.52599245585101 0.6
auc, f1, accu, accu_bl, precision, recall=  0.6402855310507107 0.5042973522820507 0.7563761366156576 0.5629881091910904 0.10657797189646334 0.3268053855569155
[[6554 1647]
 [ 550  267]]


In [81]:
sc_xgb # This is without attaching any of the other features in!!

(0.6402855310507107,
 0.52599245585101,
 0.7563761366156576,
 0.5629881091910904,
 0.10657797189646334,
 0.3268053855569155)

In [116]:
era_features = pd.read_csv('era_features.csv', index_col=0)
era_features['grid_id'] = list(zip(era_features['lat'],era_features['long']))
df_era = era_features.drop(['lat','long'],axis = 1)
xy_df_sub_2 = pd.merge(df_era, xy_df, on=['grid_id','year'], how='right')

In [117]:
#construct x,y train and test set 
x = xy_df_sub_2.drop(xy_df_sub_2.filter(regex='target').columns, axis=1)#drop target col 
x = x.select_dtypes(['number'])#drop index col
y = xy_df_sub_2.filter(regex='target') #filter to cols containing target 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print(x_train.shape) #x_train contains stat features only 


y_pred, y_pred_prob = m.run_xgb(x_train, y_train, x_test)
sc_xgb = m.get_scores_clf(y_test, y_pred_prob)

(21040, 133)
running xgb...
Train AUC:  0.8505983512002224
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.542495211421765 0.6
auc, f1, accu, accu_bl, precision, recall=  0.6446646429511164 0.5134967167232402 0.8112663561765359 0.5573506052117416 0.10694301893497406 0.24724602203182375
[[7114 1087]
 [ 615  202]]


In [118]:
sc_xgb #Somehow ERA5 features gave worse performance? (which ERA features are we choosing?) (are we sure adding all of the ,helps

(0.6446646429511164,
 0.542495211421765,
 0.8112663561765359,
 0.5573506052117416,
 0.10694301893497406,
 0.24724602203182375)

In [131]:
xy_df_sub_3 =  pd.merge(df_nlp, xy_df_sub_2, on='grid_id', how='right')

In [132]:
#construct x,y train and test set 
x = xy_df_sub_3.drop(xy_df_sub_3.filter(regex='target').columns, axis=1)#drop target col 
x = x.select_dtypes(['number'])#drop index col
y = xy_df_sub_3.filter(regex='target') #filter to cols containing target 

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print(x_train.shape) #x_train contains stat features only 


y_pred, y_pred_prob = m.run_xgb(x_train, y_train, x_test)
sc_xgb = m.get_scores_clf(y_test, y_pred_prob)

(21040, 165)
running xgb...
Train AUC:  0.8505983512002224
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.542495211421765 0.6
auc, f1, accu, accu_bl, precision, recall=  0.6446646429511164 0.5134967167232402 0.8112663561765359 0.5573506052117416 0.10694301893497406 0.24724602203182375
[[7114 1087]
 [ 615  202]]


In [133]:
sc_xgb

(0.6446646429511164,
 0.542495211421765,
 0.8112663561765359,
 0.5573506052117416,
 0.10694301893497406,
 0.24724602203182375)

In [110]:
def create_nlp_modality(name = 'nlp_cls_transfer.pkl'):
    """
    Helper function that let's us load the NLP featuer
    
    Input = String of the name of the NLP file 
    """

    #load processed nlp features: 
    #note: you can load other nlp features too, they all start with 'nlp_*' 
    file = open('../../data/'+name, 'rb') 
    # load file
    df_nlp = pickle.load(file)
    # close the file
    file.close()
    df_nlp.shape

    #correct formatting for df_nlp 
    df_nlp = string_to_tuple(df_nlp, 'grid_id')
    
    #drop text, location and label columns 
    df_nlp = df_nlp.drop(['location','txt'], axis=1)
    
    return df_nlp

def string_to_tuple(df, col): 
    try: 
        df[col] = df.apply(lambda row: eval(row[col]), axis=1) 
    except: 
        'error converting to tuple'
    return df

def attach_target(x_df, y_master, disaster, next_n): 
    y = y_master.copy()
    #shift years
    y['year'] = y['year'] - next_n
    #keep for particular disaster 
    y = y[['grid_id','year',disaster+'_bin']]
    # Rename into target
    y = y.rename(columns={disaster +'_bin': 'target_' + disaster + '_'+ str(next_n)})
    xy_df = pd.merge(x_df, y, on = ['grid_id','year'], how='inner')
    return xy_df


def data_production(df_yrly,n_pred,y_master,modality,nlp_name):
    '''
    Helper Functions to deal with modality
    '''
    
    #select to those id pass the filtering criteria 
    x_df = df_yrly.loc[df_yrly['grid_id'].isin(grid_id_ls)]
    #interpolate missing years to have no flood 
    x_df = pd.merge(idd, x_df, on=['grid_id','year'], how='left').fillna(0)
    
    print('length of x_df', len(x_df))

    
    xy_df_sub = attach_target(x_df, y_master, 'flood', n_pred)
    print('length of xy_df', len(xy_df))

    print('length of xy_df', len(xy_df_sub))
    print('imbalance', xy_df_sub.filter(regex='target').sum()/len(xy_df_sub))
    
    name = "statistical"
    for mod in modality:
        name = name + "_" +mod
        

    if "era5" in modality:
        # we have to attach at this step
        era_features = pd.read_csv('era_features.csv', index_col=0)
        era_features['grid_id'] = list(zip(era_features['lat'],era_features['long']))
        df_era = era_features.drop(['lat','long'],axis = 1)
        xy_df_sub = pd.merge(df_era, xy_df_sub, on=['grid_id','year'], how='right')
    
    #merge with NLP 
    if "nlp" in modality:
        df_nlp = create_nlp_modality(nlp_name)
        xy_df_sub = pd.merge(xy_df_sub, df_nlp,on=['grid_id'], how='left')
        print('shape of xy_df with nlp features now included', xy_df_sub.shape)
        name = name + nlp_name


    #construct x,y train and test set 
    x = xy_df_sub.drop(xy_df_sub.filter(regex='target').columns, axis=1)#drop target col 
    x = x.select_dtypes(['number'])#drop index col
    y = xy_df_sub.filter(regex='target') #filter to cols containing target 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
    print(x_train.shape) 
    
    
    #Gradient Boosting Running
    results={}
    y_pred, y_pred_prob = m.run_xgb(x_train, y_train, x_test)
    results[name] = m.get_scores_clf(y_test, y_pred_prob)
    
    return pd.DataFrame(results)

In [111]:
modality_combi =  [ 
    [],
    ['era5'],
    ['nlp'],
    ['era5','nlp']
]
nlp_names = ['nlp_cls_transfer.pkl','nlp_avg.pkl','nlp_cls_finetune.pkl','nlp_cls_transfer.pkl','nlp_cls.pkl']
horizons = [i for i in range(1,5)]
# horizons = [1]

In [112]:
result_dict = {} 

In [113]:
for n_pred in horizons:
    result_dict[n_pred] = []
    for modality in modality_combi:
        
        if 'nlp' in modality:
            for nlp_name in nlp_names:
                result_dict[n_pred].append(data_production(df_yrly,n_pred,y_master,modality,nlp_name))
            
        else:
            result_dict[n_pred].append(data_production(df_yrly,n_pred,y_master,modality,''))

length of x_df 31640
length of xy_df 30058
length of xy_df 30849
imbalance target_flood_1    0.090181
dtype: float64
(21594, 25)
running xgb...
Train AUC:  0.7016140105763371
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5497519370876384 0.6
auc, f1, accu, accu_bl, precision, recall=  0.6451804244106573 0.5124804738130077 0.8022690437601296 0.5714710092390171 0.11443726592130024 0.2882147024504084
[[7178 1220]
 [ 610  247]]
length of x_df 31640
length of xy_df 30058
length of xy_df 30849
imbalance target_flood_1    0.090181
dtype: float64
(21594, 133)
running xgb...
Train AUC:  0.865804017529067
{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'scale_pos_weight': 10}
maximum f1 score, thres 0.5576934105637437 0.6
auc, f1, accu, accu_bl, precision, recall=  0.6566163027647578 0.5208983775810285 0.8314424635332253 0.5665905617912583 0.1149940196085657 0.24154025670945156
[[7488  910]
 [ 650  207]]
length of x_df 3

In [114]:
concat_list = []
for key in result_dict:
    concat_list.append(pd.concat(result_dict[key],axis= 1))
lol = pd.concat(concat_list)

lol.to_csv('results_new.csv')